In [10]:
import pandas as pd
from pymongo import MongoClient
import matplotlib.pyplot as plt
import seaborn as sns

# 1. MongoDB 연결
client = MongoClient("mongodb://localhost:27017/")
db = client["f1_bigdata_db"]
collection_telemetry = db["telemetry_raw"]
collection_races = db["race_meta"]

def check_db_status():
    """
    MongoDB의 현재 저장 용량과 데이터 개수를 출력하는 함수
    """
    print("\n===== 🗄️ MongoDB 저장 현황 리포트 =====")

    # 1. 데이터 개수 확인
    telemetry_count = collection_telemetry.count_documents({})
    race_count = collection_races.count_documents({})

    # 2. DB 용량 확인 (dbstats 명령어)
    stats = db.command("dbstats")

    # 바이트(Byte) 단위를 기가바이트(GB)로 변환
    # storageSize: 압축되어 실제 디스크를 차지하는 용량
    # dataSize: 압축 전 순수 데이터 용량
    storage_size_gb = stats['storageSize'] / (1024 ** 3)
    data_size_gb = stats['dataSize'] / (1024 ** 3)

    print(f"✅ 수집된 레이스 수: {race_count} 개")
    print(f"✅ 수집된 텔레메트리 데이터(행): {telemetry_count:,} 개")
    print(f"📦 디스크 점유 용량 (Storage): {storage_size_gb:.2f} GB")
    print(f"📊 실제 데이터 용량 (Raw): {data_size_gb:.2f} GB")
    print("======================================\n")

def visualize_2023_race_test(year, round_num, driver_code):
    """
    특정 라운드, 특정 드라이버의 데이터를 시각화하여 데이터 무결성 확인
    """

    # MongoDB 쿼리
    query = {
        "Year": year,
        "Round": round_num,
        "Driver": driver_code
    }

    # 필요한 컬럼만 가져오기 (메모리 절약)
    projection = {"_id": 0, "Speed": 1, "Distance": 1}

    # 데이터 로드
    cursor = collection_telemetry.find(query, projection)
    df = pd.DataFrame(list(cursor))

    if df.empty:
        print("⚠️ 경고: 해당 조건의 데이터가 없습니다. (아직 수집되지 않았거나 조건 오류)")
        return

    # 거리순 정렬 (그래프가 꼬이지 않게)
    df = df.sort_values(by="Distance")

    # 시각화 (속도 그래프)
    plt.figure(figsize=(15, 5))

    # 데이터가 너무 많으면 초반 10,000개 포인트(약 1~2랩)만 찍어봅니다.
    subset = df.iloc[:10000] if len(df) > 10000 else df

    sns.lineplot(data=subset, x="Distance", y="Speed", linewidth=1, color='royalblue')

    plt.title(f"Speed Trace Check - 2023 Round {round_num} (Driver: {driver_code})", fontsize=14)
    plt.xlabel("Distance (m)")
    plt.ylabel("Speed (km/h)")
    plt.grid(True, alpha=0.5)
    plt.show()
    print("✨ 시각화 완료! 데이터가 정상적으로 보입니다.")

# --- 실행부 ---
if __name__ == "__main__":
    # 1단계: DB 용량 체크
    check_db_status()
    #
    # # 2단계: 데이터가 조금이라도 있다면 시각화 시도
    # visualize_2023_race_test(2019, 3, "33") # 2023년 1라운드, 베르스타펜(1)


===== 🗄️ MongoDB 저장 현황 리포트 =====
✅ 수집된 레이스 수: 149 개
✅ 수집된 텔레메트리 데이터(행): 62,337,368 개
📦 디스크 점유 용량 (Storage): 3.36 GB
📊 실제 데이터 용량 (Raw): 13.76 GB



In [1]:
import pandas as pd
from pymongo import MongoClient

# 1. MongoDB 연결
client = MongoClient("mongodb://localhost:27017/")
db = client["f1_bigdata_db"]
col_telemetry = db["telemetry_raw"]
col_races = db["race_meta"]

def detailed_data_audit():
    print("🕵️‍♀️ [F1 데이터 정밀 감사] DB 전수 조사를 시작합니다...")
    print("   (데이터 양에 따라 1~2분 정도 소요될 수 있습니다)\n")

    # --- 1. 기준 정보 로드 (Race Meta) ---
    # 우리가 수집 '해야 할' 모든 경기 목록
    meta_cursor = col_races.find({}, {"_id": 0, "year": 1, "round": 1, "event_name": 1})
    df_meta = pd.DataFrame(list(meta_cursor)).rename(columns={'year': 'Year', 'round': 'Round', 'event_name': 'Event'})

    if df_meta.empty:
        print("❌ [치명적 오류] 'race_meta' 컬렉션이 비어있습니다. 수집 코드를 다시 확인해주세요.")
        return

    # --- 2. 실제 데이터 집계 (Telemetry Aggregation) ---
    # MongoDB가 직접 계산하게 하여 속도를 높입니다.
    pipeline = [
        {
            "$group": {
                "_id": {"Year": "$Year", "Round": "$Round"},
                "Unique_Drivers": {"$addToSet": "$Driver"}, # 드라이버 목록 (중복 제거)
                "Total_Points": {"$sum": 1}                # 텔레메트리 포인트 수
            }
        },
        {
            "$project": {
                "_id": 0,
                "Year": "$_id.Year",
                "Round": "$_id.Round",
                "Driver_Count": {"$size": "$Unique_Drivers"}, # 드라이버 수 카운트
                "Telemetry_Count": "$Total_Points"
            }
        }
    ]

    telemetry_cursor = col_telemetry.aggregate(pipeline)
    df_actual = pd.DataFrame(list(telemetry_cursor))

    # --- 3. 비교 분석 ---
    # 메타 데이터(계획)에 실제 데이터(실적)를 결합
    df_audit = pd.merge(df_meta, df_actual, on=['Year', 'Round'], how='left')

    # 결측치 처리 (수집 안 된 경기는 0으로)
    df_audit['Driver_Count'] = df_audit['Driver_Count'].fillna(0).astype(int)
    df_audit['Telemetry_Count'] = df_audit['Telemetry_Count'].fillna(0).astype(int)

    # 정렬
    df_audit = df_audit.sort_values(by=['Year', 'Round'])

    # --- 4. 결과 리포트 생성 ---

    # [기준 1] 아예 수집 안 된 경기
    missing = df_audit[df_audit['Telemetry_Count'] == 0]

    # [기준 2] 수집은 됐으나 드라이버 수가 너무 적은 경기 (15명 미만)
    # (2014~2016년 데이터는 원래 적을 수도 있으나 확인 필요)
    low_drivers = df_audit[(df_audit['Telemetry_Count'] > 0) & (df_audit['Driver_Count'] < 15)]

    print("="*60)
    print(f"📊 전체 요약 보고서 ({df_audit['Year'].min()} ~ {df_audit['Year'].max()})")
    print("="*60)
    print(f"• 총 예정 경기 수 : {len(df_audit)} 개")
    print(f"• 수집 완료 경기 수 : {len(df_audit) - len(missing)} 개")
    print(f"• 데이터 누락 경기 수 : {len(missing)} 개")
    print(f"• 총 텔레메트리 데이터 : {df_audit['Telemetry_Count'].sum():,} 행 (Rows)")
    print("-" * 60)

    # 시즌별 상세 현황 출력
    print("\n📅 [시즌별 수집 현황]")
    season_summary = df_audit.groupby('Year').agg({
        'Event': 'count',
        'Driver_Count': 'mean',
        'Telemetry_Count': 'sum'
    }).rename(columns={'Event': 'Rounds', 'Driver_Count': 'Avg.Drivers', 'Telemetry_Count': 'Total_Data'})
    print(season_summary)

    # 문제점 보고
    print("\n" + "="*60)
    print("🚨 [상세 문제점 리포트]")

    if not missing.empty:
        print(f"\n❌ 1. 데이터가 아예 없는 경기 ({len(missing)}건):")
        # 보기 좋게 출력
        for _, row in missing.iterrows():
            print(f"   - {row['Year']} R{row['Round']} : {row['Event']}")
        print("   👉 해결책: 해당 연도만 다시 수집 코드 실행")
    else:
        print("\n✅ 1. 누락된 경기가 없습니다. (모든 라운드 데이터 존재)")

    if not low_drivers.empty:
        print(f"\n⚠️ 2. 드라이버 수가 부족한 경기 (<15명) ({len(low_drivers)}건):")
        for _, row in low_drivers.iterrows():
            print(f"   - {row['Year']} R{row['Round']} {row['Event']} : 드라이버 {row['Driver_Count']}명")
        print("   👉 참고: 2014~2016년은 센서 데이터가 원래 부족할 수 있습니다.")
    else:
        print("\n✅ 2. 모든 경기의 드라이버 데이터가 충분합니다.")

    print("="*60)
    print("\n[최종 진단]")
    if missing.empty:
        print("🎉 축하합니다! 데이터 수집이 완벽하게 마무리되었습니다.")
    else:
        print("🔥 일부 누락된 데이터가 있습니다. 위 리스트를 확인하여 보충해주세요.")

# --- 실행 ---
if __name__ == "__main__":
    detailed_data_audit()

🕵️‍♀️ [F1 데이터 정밀 감사] DB 전수 조사를 시작합니다...
   (데이터 양에 따라 1~2분 정도 소요될 수 있습니다)

📊 전체 요약 보고서 (2018 ~ 2024)
• 총 예정 경기 수 : 149 개
• 수집 완료 경기 수 : 148 개
• 데이터 누락 경기 수 : 1 개
• 총 텔레메트리 데이터 : 60,133,959 행 (Rows)
------------------------------------------------------------

📅 [시즌별 수집 현황]
      Rounds  Avg.Drivers  Total_Data
Year                                 
2018      21    19.000000     8277891
2019      21    20.000000     9177760
2020      17    20.000000     7020645
2021      22    19.909091     9022564
2022      22    19.909091     8442895
2023      22    19.909091     8802481
2024      24    19.916667     9389723

🚨 [상세 문제점 리포트]

❌ 1. 데이터가 아예 없는 경기 (1건):
   - 2018 R2 : Bahrain Grand Prix
   👉 해결책: 해당 연도만 다시 수집 코드 실행

✅ 2. 모든 경기의 드라이버 데이터가 충분합니다.

[최종 진단]
🔥 일부 누락된 데이터가 있습니다. 위 리스트를 확인하여 보충해주세요.


테스트 시각화 : 2023 R1 바레인GP 1위 VER과 2위 PER의 데이터 비교

In [4]:
import pandas as pd
from pymongo import MongoClient
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# 1. MongoDB 연결
client = MongoClient("mongodb://localhost:27017/")
db = client["f1_bigdata_db"]
col_telemetry = db["telemetry_raw"]

# --- 설정 ---
YEAR = 2023
ROUND = 1
DRIVER_1 = "1"  # Max Verstappen
DRIVER_2 = "11" # Sergio Perez

def load_driver_data(driver_id):
    print(f"📥 드라이버 {driver_id} 데이터 로딩 중...")
    cursor = col_telemetry.find(
        {"Year": YEAR, "Round": ROUND, "Driver": driver_id},
        {"_id": 0, "Speed": 1, "Throttle": 1, "Brake": 1, "nGear": 1, "Distance": 1}
    )
    df = pd.DataFrame(list(cursor))
    if df.empty: return None

    # 데이터 정렬 및 1랩(5.4km) 추출
    df = df.sort_values(by="Distance")
    return df[df['Distance'] <= 5412]

def create_interactive_dashboard():
    df1 = load_driver_data(DRIVER_1)
    df2 = load_driver_data(DRIVER_2)

    if df1 is None or df2 is None:
        print("데이터 로딩 실패")
        return

    # Plotly 서브플롯 생성 (4개 행)
    fig = make_subplots(
        rows=4, cols=1, shared_xaxes=True,
        vertical_spacing=0.05,
        subplot_titles=("Speed (km/h)", "Throttle (%)", "Brake (On/Off)", "Gear"),
        row_heights=[0.4, 0.2, 0.2, 0.2]
    )

    # 색상 설정
    color_d1 = '#1f77b4' # 파랑 (VER)
    color_d2 = '#d62728' # 빨강 (PER)

    # 1. Speed Trace
    fig.add_trace(go.Scatter(x=df1['Distance'], y=df1['Speed'], mode='lines', name=f'VER #{DRIVER_1}', line=dict(color=color_d1, width=2)), row=1, col=1)
    fig.add_trace(go.Scatter(x=df2['Distance'], y=df2['Speed'], mode='lines', name=f'PER #{DRIVER_2}', line=dict(color=color_d2, width=2, dash='dash')), row=1, col=1)

    # 2. Throttle
    fig.add_trace(go.Scatter(x=df1['Distance'], y=df1['Throttle'], mode='lines', name='VER Throttle', line=dict(color=color_d1), showlegend=False), row=2, col=1)
    fig.add_trace(go.Scatter(x=df2['Distance'], y=df2['Throttle'], mode='lines', name='PER Throttle', line=dict(color=color_d2, dash='dash'), showlegend=False), row=2, col=1)

    # 3. Brake
    # Brake는 Boolean일 수 있으므로 int로 변환
    brake1 = df1['Brake'].astype(int)
    brake2 = df2['Brake'].astype(int)

    fig.add_trace(go.Scatter(x=df1['Distance'], y=brake1, mode='lines', name='VER Brake', line=dict(color=color_d1), fill='tozeroy', showlegend=False), row=3, col=1)
    fig.add_trace(go.Scatter(x=df2['Distance'], y=brake2, mode='lines', name='PER Brake', line=dict(color=color_d2, dash='dash'), showlegend=False), row=3, col=1)

    # 4. Gear
    fig.add_trace(go.Scatter(x=df1['Distance'], y=df1['nGear'], mode='lines', name='VER Gear', line=dict(color=color_d1, shape='hv'), showlegend=False), row=4, col=1)
    fig.add_trace(go.Scatter(x=df2['Distance'], y=df2['nGear'], mode='lines', name='PER Gear', line=dict(color=color_d2, dash='dash', shape='hv'), showlegend=False), row=4, col=1)

    # 레이아웃 꾸미기
    fig.update_layout(
        title=f"<b>F1 {YEAR} Round {ROUND} Telemetry Analysis</b> <br>Max Verstappen vs Sergio Perez",
        height=1000,  # 전체 높이
        hovermode="x unified", # 마우스 오버 시 x축 기준으로 모든 값 표시
        template="plotly_white",
        xaxis4_title="Distance (meters)"
    )

    # Y축 범위 설정
    fig.update_yaxes(title_text="km/h", row=1, col=1)
    fig.update_yaxes(title_text="%", range=[-5, 105], row=2, col=1)
    fig.update_yaxes(title_text="On/Off", range=[-0.1, 1.1], tickvals=[0, 1], ticktext=["Off", "On"], row=3, col=1)
    fig.update_yaxes(title_text="Gear", range=[0, 9], row=4, col=1)

    # HTML 파일로 저장
    output_file = "F1_Interactive_Analysis.html"
    fig.write_html(output_file)
    print(f"\n✅ 생성 완료! '{output_file}' 파일을 브라우저에서 여세요.")

# --- 실행 ---
if __name__ == "__main__":
    create_interactive_dashboard()

📥 드라이버 1 데이터 로딩 중...
📥 드라이버 11 데이터 로딩 중...

✅ 생성 완료! 'F1_Interactive_Analysis.html' 파일을 브라우저에서 여세요.
